In [1]:
from PA0 import PA0

In [2]:
import dask.dataframe as dd
import pandas as pd
from dask.distributed import Client
import json

In [3]:
{"number_products_rated": {"count": 996.0, "mean": 1.0, "std": 0.07, "min": 1.0, "25%": 1.0, "50%": 1.0, "75%": 1.0, "max": 3.0}, 
 "avg_ratings": {"count": 996.0, "mean": 4.16, "std": 1.25, "min": 1.0, "25%": 4.0, "50%": 5.0, "75%": 5.0, "max": 5.0}, 
 "reviewing_since": {"count": 996.0, "mean": 2011.63, "std": 3.01, "min": 1999.0, "25%": 2011.0, "50%": 2013.0, "75%": 2013.0, "max": 2014.0}, 
 "helpful_votes": {"count": 996.0, "mean": 1.96, "std": 8.81, "min": 0.0, "25%": 0.0, "50%": 0.0, "75%": 1.0, "max": 152.0}, 
 "total_votes": {"count": 996.0, "mean": 2.72, "std": 10.16, "min": 0.0, "25%": 0.0, "50%": 0.0, "75%": 2.0, "max": 157.0}}

{'number_products_rated': {'count': 996.0,
  'mean': 1.0,
  'std': 0.07,
  'min': 1.0,
  '25%': 1.0,
  '50%': 1.0,
  '75%': 1.0,
  'max': 3.0},
 'avg_ratings': {'count': 996.0,
  'mean': 4.16,
  'std': 1.25,
  'min': 1.0,
  '25%': 4.0,
  '50%': 5.0,
  '75%': 5.0,
  'max': 5.0},
 'reviewing_since': {'count': 996.0,
  'mean': 2011.63,
  'std': 3.01,
  'min': 1999.0,
  '25%': 2011.0,
  '50%': 2013.0,
  '75%': 2013.0,
  'max': 2014.0},
 'helpful_votes': {'count': 996.0,
  'mean': 1.96,
  'std': 8.81,
  'min': 0.0,
  '25%': 0.0,
  '50%': 0.0,
  '75%': 1.0,
  'max': 152.0},
 'total_votes': {'count': 996.0,
  'mean': 2.72,
  'std': 10.16,
  'min': 0.0,
  '25%': 0.0,
  '50%': 0.0,
  '75%': 2.0,
  'max': 157.0}}

In [4]:
# import timeit

# start = timeit.default_timer()

# PA0('user_reviews_Release.csv')

# stop = timeit.default_timer()

# print('Time: ', stop - start)  

In [5]:
# df = dd.read_csv('small_dataset.csv').drop(['reviewText','summary','unixReviewTime','reviewerName'],axis=1)

In [6]:
# df.compute()

In [7]:
# df['reviewing_since']=df['reviewTime'].str.slice(start=-4).astype('int16')
# df['helpful_votes']=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[0]).astype('int16')
# df['total_votes']=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[1]).astype('int16')

In [8]:
# ddf = df.repartition(10)

In [9]:
# %timeit -n 5 -r 2 PA0('small_dataset.csv')

In [10]:

# df.map_partitions(lambda x: x.groupby('reviewerID').agg({"asin": "count",'overall':'mean','reviewing_since':'min','helpful_votes':'sum','total_votes':'sum'})).describe().compute()


In [19]:
df = dd.read_csv('small_dataset.csv').drop(['reviewText','summary','unixReviewTime','reviewerName'],axis=1)

In [20]:
df.compute()

,reviewerID,asin,helpful,overall,reviewTime
0,A2T0RJ91B0PQ03,B0016CRVLW,"[0, 0]",1.0,"05 21, 2014"
1,A3TYW0XA8HSGWB,B00EKR5S0Q,"[0, 0]",5.0,"04 27, 2014"
2,A2CME0TQU2IVVB,B001AUPJVO,"[1, 1]",5.0,"12 19, 2012"
3,A2E5IDLX7R388S,B000055Y57,"[0, 0]",5.0,"06 10, 2014"
4,A3CIEMYUGV6ZMR,0545265355,"[0, 0]",5.0,"04 11, 2012"
...,...,...,...,...,...
994,A2AZAO0FLD1NN6,0984032827,"[1, 1]",5.0,"10 29, 2012"
995,A303MONZTW2M4Y,B00009MGF7,"[15, 16]",4.0,"09 8, 2003"
996,A35G0ZAEJQA14R,B007TVUZB2,"[1, 2]",5.0,"10 4, 2012"
997,AWAI3GPWL5Y9H,B006WQE5S4,"[92, 106]",1.0,"07 25, 2012"


In [21]:
df.dtypes

reviewerID     object
asin           object
helpful        object
overall       float64
reviewTime     object
dtype: object

In [16]:
# df.map_partitions(lambda df: df.assign(reviewing_since=df['reviewTime'].str.slice(start=-4)),meta={'reviewing_since':'int16'})

In [ ]:
df['helpful'].str[1:-1].str.split(',').map(lambda x: x[1]).compute()

In [ ]:
df['helpful'].str[1:-1].str.split(',').map(lambda x: x[-1]).compute()

In [42]:
def assign_col(df):
    df.assign(reviewing_since=df['reviewTime'].str.slice(start=-4).astype('int16'),
                                       total_votes=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[1]).astype('int16'),
                                       helpful_votes=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[0]).astype('int16'),
                                       number_products_rated=df['asin'],
                                       avg_ratings=df['overall'])
                                       

In [43]:
ddf = df.map_partitions(assign_col,meta={'reviewing_since':'int16','total_votes':'int16','helpful_votes':'int16','number_products_rated':'int16','avg_ratings':'int16'})

In [ ]:
ddf.persist().describe().compute()

AttributeError: 'NoneType' object has no attribute 'sum'

In [22]:
res=df.map_partitions(lambda df: df.assign(reviewing_since=df['reviewTime'].str.slice(start=-4),
                                       total_votes=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[1]),
                                       helpful_votes=df['helpful'].str[1:-1].str.split(',').map(lambda x: x[0]),
                                       number_products_rated=df['asin'],
                                       avg_ratings=df['overall'],
                                       reviewerID=df['reviwerID']),
                                       meta={'reviewing_since':'int16','total_votes':'int16','helpful_votes':'int16','number_products_rated':'int16','avg_ratings':'int16','reviewerID':'object'})

In [29]:
res.head()

KeyError: 'reviwerID'

In [34]:
res.groupby('reviewerID').agg({"number_products_rated": "count",'avg_ratings':'mean','reviewing_since':'min','helpful_votes':'sum','total_votes':'sum'}).describe().compute()

KeyError: 'reviwerID'

In [ ]:
# res.head()

In [ ]:
# ddf

In [ ]:
# ddf.compute()

In [ ]:
# df5 = df['helpful'].repartition(10)
# for i in range(df5.npartitions):
#     print(df5.partitions[i].compute())

In [ ]:
# import timeit

# start = timeit.default_timer()

# PA0('user_reviews_Release.csv')

# stop = timeit.default_timer()

# print('Time: ', stop - start)  